In [1]:
import pandas as pd
import numpy as np
import pickle 
import os

In [24]:
# Step 1: Open the file in read-binary mode
with open('data.pkl', 'rb') as file:
    # Step 2: Load the data from the file
    data = pickle.load(file)

# Now, `data` contains the deserialized Python object
print(data)


0        JAN JAGRAN TIMES
1        JAGRAN CITY PLUS
2         SAMPURNA JAGRAN
3           DAINIK JAGRAN
4           VISHWA JAGRAN
               ...       
21394        KAIWART AWAZ
21395     SARBAHARAR AWAZ
21396      SHRAMIKER AWAZ
21397          SOBAR AWAZ
21398        AWAZ AAP TAK
Name: Title Name, Length: 10790, dtype: object


In [15]:
type(data)


pandas.core.series.Series

In [25]:
data = pd.DataFrame(data)
data = data.drop_duplicates().dropna()

In [26]:
data.shape
data

,Title Name
0,JAN JAGRAN TIMES
1,JAGRAN CITY PLUS
2,SAMPURNA JAGRAN
3,DAINIK JAGRAN
4,VISHWA JAGRAN
...,...
21394,KAIWART AWAZ
21395,SARBAHARAR AWAZ
21396,SHRAMIKER AWAZ
21397,SOBAR AWAZ


In [27]:
data.rename(columns={'Title Name': 'original'}, inplace=True)

data['title'] = data['original'].str.lower().apply(lambda x: ' '.join(sorted(x.split())))


In [28]:
data

,original,title
0,JAN JAGRAN TIMES,jagran jan times
1,JAGRAN CITY PLUS,city jagran plus
2,SAMPURNA JAGRAN,jagran sampurna
3,DAINIK JAGRAN,dainik jagran
4,VISHWA JAGRAN,jagran vishwa
...,...,...
21394,KAIWART AWAZ,awaz kaiwart
21395,SARBAHARAR AWAZ,awaz sarbaharar
21396,SHRAMIKER AWAZ,awaz shramiker
21397,SOBAR AWAZ,awaz sobar


In [7]:
!pip install langchain

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # Import Document class

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Assuming your DataFrame `data` has a column 'title' containing the titles or documents
# Create a list of Document objects
documents = [Document(page_content=row['title']) for index, row in data.iterrows()]

# Split the documents
split_docs = text_splitter.split_documents(documents)

In [30]:
# Extract content from split_docs
texts = [doc.page_content for doc in split_docs]


In [10]:
!pip install langchain-ollama 

In [9]:
import sys
!{sys.executable} -m pip install "transformers[sentencepiece]"


In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")

ValueError: Unrecognized model in sarvamai/sarvam-1. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, encodec, encoder-decoder, ernie, ern

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings.base import Embeddings

class SarvamEmbeddings(Embeddings):
    def __init__(self, model_name="sarvamai/sarvam-1"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Ensure padding token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        inputs = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
            hidden = outputs.last_hidden_state  # (batch, seq_len, hidden_dim)
            embeddings = hidden.mean(dim=1)     # mean pooling
            return embeddings.cpu().tolist()

    def embed_query(self, text: str) -> list[float]:
        return self.embed_documents([text])[0]



embeddings = SarvamEmbeddings()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
res = embeddings.embed_query("The Jagran Times")
print(len(res)) # checking the dimension of the embeddings so that the pinecone vector db can be configured accordingly 

2048


In [32]:
from dotenv import load_dotenv
load_dotenv()
import os
api_key = os.getenv("PINECONE_API_KEY2")

In [14]:
!pip install pinecone

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.


In [5]:
from pinecone import Pinecone
index_name = "sarvam"
pc = Pinecone(api_key=api_key)
index = pc.Index(index_name)
# Setting up the pinecone index the index name in this case is llama 

In [16]:
!pip install pinecone-text

In [6]:
import nltk
nltk.download('stopwords' )
nltk.download('punkt' )

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Debanjan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Debanjan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk
nltk.data.find('tokenizers/punkt')


FileSystemPathPointer('C:\\Users\\Debanjan\\nltk_data\\tokenizers\\punkt')

In [15]:
from pinecone_text.sparse import BM25Encoder


import nltk
nltk.download('punkt')  # force correct download
nltk.download('stopwords')

# Initialize the BM25 encoder
encoder = BM25Encoder().default()

# Your data and BM25 encoding process
# Ensure the data column is converted to a list of strings
corpus = data['title'].tolist()  # Convert the column to a list

# Fit the encoder
encoder.fit(corpus)

# Save the encoded data
encoder.dump("document.json")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Debanjan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Debanjan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/10790 [00:00<?, ?it/s]

In [16]:
encoded_docs = BM25Encoder().load("document.json")

In [17]:

alpha= 0.7


In [24]:
!pip install langchain-community

  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
Using cached langchain_community-0.3.24-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


In [18]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(index=index, sparse_encoder=encoded_docs, embeddings=embeddings , top_k=30 , alpha = 0.8)

In [19]:
retriever

PineconeHybridSearchRetriever(embeddings=<__main__.SarvamEmbeddings object at 0x000001DF09ECBC80>, sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001E2A080C620>, index=<pinecone.db_data.index.Index object at 0x000001DF146347A0>, top_k=30, alpha=0.8)

In [20]:
retriever.add_texts(
    corpus
)

  0%|          | 0/338 [00:00<?, ?it/s]

In [ ]:
input_text = "Denik Micheal jagran"

In [ ]:
documents=retriever.invoke(input_text)

In [ ]:
documents

[Document(metadata={'score': 0.73353374}, page_content='krisak jagran'),
 Document(metadata={'score': 0.711413}, page_content='jan jagran'),
 Document(metadata={'score': 0.692397237}, page_content='shosit jagran'),
 Document(metadata={'score': 0.676488638}, page_content='swadeep jagran'),
 Document(metadata={'score': 0.674240828}, page_content='pal jagran'),
 Document(metadata={'score': 0.671086073}, page_content='maruthar jagran'),
 Document(metadata={'score': 0.669752598}, page_content='satat jagran'),
 Document(metadata={'score': 0.669108272}, page_content='vasundhara jagran'),
 Document(metadata={'score': 0.666702628}, page_content='chaitanya jagran'),
 Document(metadata={'score': 0.663770795}, page_content='brahman jagran'),
 Document(metadata={'score': 0.661911488}, page_content='kaurav jagran'),
 Document(metadata={'score': 0.661866486}, page_content='india samikaran'),
 Document(metadata={'score': 0.660538912}, page_content='kishan jagran'),
 Document(metadata={'score': 0.65691

In [ ]:
from rapidfuzz import fuzz





# Perform fuzzy matching
results = []
for doc in documents:
    similarity_score = fuzz.ratio(input_text.lower(), doc.page_content.lower())
    results.append({'page_content': doc.page_content, 'similarity_score': similarity_score})

# Sort the results by similarity score in descending order
sorted_results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)

# Display all results with similarity scores in sorted order
for result in sorted_results:
    print(f"Title: {result['page_content']}, Similarity: {result['similarity_score']}%")


Title: audichya jagran, Similarity: 68.57142857142857%
Title: kishan jagran, Similarity: 66.66666666666667%
Title: krisak jagran, Similarity: 60.60606060606061%
Title: khatik jagran, Similarity: 60.60606060606061%
Title: pal jagran, Similarity: 60.0%
Title: nandigiram jagran, Similarity: 59.45945945945945%
Title: swadeep jagran, Similarity: 58.82352941176471%
Title: lakshya jagran, Similarity: 58.82352941176471%
Title: manohar jagran, Similarity: 58.82352941176471%
Title: sandhya jagran, Similarity: 58.82352941176471%
Title: maruthar jagran, Similarity: 57.14285714285714%
Title: sewak jagran, Similarity: 56.25%
Title: chaitanya jagran, Similarity: 55.55555555555556%
Title: shrutlekh jagran, Similarity: 55.55555555555556%
Title: kaurav jagran, Similarity: 54.54545454545454%
Title: balmik jagran, Similarity: 54.54545454545454%
Title: vasundhara jagran, Similarity: 54.054054054054056%
Title: jan jagran, Similarity: 53.333333333333336%
Title: lok jagran, Similarity: 53.333333333333336%
Tit